In [1]:
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [2]:

data_dict = pickle.load(open('./data.pickle', 'rb'))
data = data_dict["data"]
# Pad the sequences to the maximum length (in this case, let's assume max length is 84)
padded_data = pad_sequences(data, maxlen=84, padding='post', dtype='float32')
data = np.stack(padded_data)
labels = np.asarray(data_dict['labels'])

In [3]:
print(data.shape)

(660, 84)


In [4]:
X = np.array(data)
X = np.expand_dims(X, -1)
X.shape

(660, 84, 1)

In [5]:
# Step 1: Convert letters to integers
label_map = {char: idx for idx, char in enumerate('ABCDEFG')}
numeric_labels = np.array([label_map[label] for label in labels])

# Step 2: Convert numeric labels to categorical format
y = to_categorical(numeric_labels).astype(int)

print(y.shape)

(660, 7)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
print(X_train.shape)
print(X_test.shape) 
print(y_train.shape)
print(y_test.shape)

(627, 84, 1)
(33, 84, 1)
(627, 7)
(33, 7)


In [15]:
from tensorflow.keras.callbacks import TensorBoard
import os

# Define TensorBoard callback
log_dir = os.path.join('Samvaad_Logs')
tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization

# Define the model
model = Sequential()

# First LSTM layer with dropout and batch normalization
model.add(LSTM(256, return_sequences=True, input_shape=(84, 1)))
model.add(Dropout(0.4))  # Dropout for regularization
model.add(BatchNormalization())  # Batch normalization for stable learning

# Second LSTM layer with dropout and batch normalization
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

# Third LSTM layer with dropout and batch normalization
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

# Fourth LSTM layer with dropout and batch normalization
model.add(LSTM(32))
model.add(Dropout(0.4))
model.add(BatchNormalization())

# Fully connected dense layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))  # Dropout for regularization

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))  # Dropout for regularization

# Output layer for multi-class classification (7 classes)
model.add(Dense(7, activation='softmax'))

# Compile the model with sparse_categorical_crossentropy
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 84, 256)           264192    
                                                                 
 dropout_6 (Dropout)         (None, 84, 256)           0         
                                                                 
 batch_normalization_4 (Batc  (None, 84, 256)          1024      
 hNormalization)                                                 
                                                                 
 lstm_5 (LSTM)               (None, 84, 128)           197120    
                                                                 
 dropout_7 (Dropout)         (None, 84, 128)           0         
                                                                 
 batch_normalization_5 (Batc  (None, 84, 128)          512       
 hNormalization)                                      

In [18]:
# Fit the model with both TensorBoard and EarlyStopping callbacks
history = model.fit(
    X_train, y_train,
    epochs=2000,
    validation_data=(X_test, y_test),
    callbacks=[tb_callback]
)


Epoch 1/2000
20/20 [==============================] - 13s 166ms/step - loss: 2.2231 - accuracy: 0.1467 - val_loss: 1.9471 - val_accuracy: 0.0909
Epoch 2/2000
20/20 [==============================] - 1s 73ms/step - loss: 1.9670 - accuracy: 0.2249 - val_loss: 1.9533 - val_accuracy: 0.0909
Epoch 3/2000
20/20 [==============================] - 1s 72ms/step - loss: 1.8762 - accuracy: 0.2584 - val_loss: 1.9545 - val_accuracy: 0.3030
Epoch 4/2000
20/20 [==============================] - 2s 76ms/step - loss: 1.8120 - accuracy: 0.2998 - val_loss: 1.9546 - val_accuracy: 0.3030
Epoch 5/2000
20/20 [==============================] - 1s 71ms/step - loss: 1.6858 - accuracy: 0.3349 - val_loss: 1.9756 - val_accuracy: 0.1515
Epoch 6/2000
20/20 [==============================] - 1s 71ms/step - loss: 1.5774 - accuracy: 0.3955 - val_loss: 2.0183 - val_accuracy: 0.1212
Epoch 7/2000
20/20 [==============================] - 2s 82ms/step - loss: 1.4436 - accuracy: 0.4705 - val_loss: 2.4005 - val_accuracy: 0.12

KeyboardInterrupt: 

In [19]:
model.save('samvaad2O.h5')

In [7]:
model.load_weights('samvaad2O.h5')

In [8]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

NameError: name 'X_test' is not defined

In [11]:
accuracy_score(ytrue, yhat)

1.0

In [27]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
actions = np.array(['A', 'B', 'C', 'D', 'E', 'F', 'G'])

hands = mp_hands.Hands( static_image_mode=True, min_detection_confidence=0.1)

# Define the desired length for padding/truncation
desired_length = 84

def preprocess_data(data_aux, desired_length=84):
    if len(data_aux) < desired_length:
        data_aux = np.pad(data_aux, (0, desired_length - len(data_aux)), 'constant')
    elif len(data_aux) > desired_length:
        data_aux = data_aux[:desired_length]
    return data_aux

cap = cv2.VideoCapture(1)

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    if not ret:
        break  # Exit loop if no frame is captured
    


    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10
        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10
        
        # Use preprocess_data to handle padding
        data_aux_padded = preprocess_data(data_aux, desired_length=84)

        # Reshape data_aux_padded to (1, 84, 2) where 1 is the batch size, 84 is the timestep, and 2 is x and y coordinates
        data_aux_padded = np.array(data_aux_padded).reshape(1, 84, 1)  # Reshaping to (1, 84, 2)

        # Ensure data_aux_padded is in the right shape for model prediction
        res = model.predict(data_aux_padded)
        predicted_character = actions[np.argmax(res)]


        # Display prediction on frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)

    cv2.imshow('frame', frame)

    # Check for key press to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 40ms/step
